In [1]:
import os
import torch
from torch import nn
from d2l import torch as d2l

/home/tian/anaconda3/envs/d2l/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [2]:
#@save
d2l.DATA_HUB['glove.6b.50d'] = (d2l.DATA_URL + 'glove.6B.50d.zip',
                                '0b8703943ccdb6eb788e6f091b8946e82231bc4d')

#@save
d2l.DATA_HUB['glove.6b.100d'] = (d2l.DATA_URL + 'glove.6B.100d.zip',
                                 'cd43bfb07e44e6f27cbcc7bc9ae3d80284fdaf5a')

#@save
d2l.DATA_HUB['glove.42b.300d'] = (d2l.DATA_URL + 'glove.42B.300d.zip',
                                  'b5116e234e9eb9076672cfeabf5469f3eec904fa')

#@save
d2l.DATA_HUB['wiki.en'] = (d2l.DATA_URL + 'wiki.en.zip',
                           'c1816da3821ae9f43899be655002f6c723e91b88')

In [11]:
#@save
class TokenEmbedding:
    """Token Embedding."""
    def __init__(self, embedding_name):
        self.idx_to_token, self.idx_to_vec = self._load_embedding(
            embedding_name)
        self.unknown_idx = 0
        self.token_to_idx = {token: idx for idx, token in
                             enumerate(self.idx_to_token)}

    def _load_embedding(self, embedding_name):
        idx_to_token, idx_to_vec = ['<unk>'], []
        data_dir = d2l.download_extract(embedding_name)
        # GloVe website: https://nlp.stanford.edu/projects/glove/
        # fastText website: https://fasttext.cc/
        with open(os.path.join(data_dir, 'vec.txt'), 'r') as f:
            for line in f:
                elems = line.rstrip().split(' ')
                token, elems = elems[0], [float(elem) for elem in elems[1:]]
                # Skip header information, such as the top row in fastText
                if len(elems) > 1:
                    idx_to_token.append(token)
                    idx_to_vec.append(elems)
        idx_to_vec = [[0] * len(idx_to_vec[0])] + idx_to_vec    # +: concatenate, not adding
        return idx_to_token, torch.tensor(idx_to_vec)

    def __getitem__(self, tokens):
        indices = [self.token_to_idx.get(token, self.unknown_idx)
                   for token in tokens]
        vecs = self.idx_to_vec[torch.tensor(indices)]
        return vecs

    def __len__(self):
        return len(self.idx_to_token)

In [31]:
glove_6b50d = TokenEmbedding('glove.6b.50d')
len(glove_6b50d)

400001

In [26]:
glove_6b50d.token_to_idx['what'], glove_6b50d.idx_to_vec[103]

(103,
 tensor([ 0.4532,  0.0598, -0.1058, -0.3330,  0.7236, -0.0872, -0.6105, -0.0377,
         -0.3095,  0.2181, -0.4360,  0.4732, -0.7687, -0.2713,  1.1042,  0.5914,
          0.5696, -0.1868,  0.1487, -0.6729, -0.3467,  0.5228,  0.2296, -0.0720,
          0.9397, -2.3985, -1.3238,  0.2870,  0.7551, -0.7652,  3.3425,  0.1723,
         -0.5180, -0.8297, -0.2933, -0.5008, -0.1523,  0.0990,  0.1815, -0.1742,
         -0.4067,  0.2035, -0.0118,  0.4825,  0.0246,  0.3406, -0.0847,  0.5324,
         -0.2510,  0.6255]))

In [39]:
def get_similar_tokens(k, word, data):
    word_vector = torch.tensor(data.idx_to_vec[data.token_to_idx[word]])
    W = torch.tensor(data.idx_to_vec)
    # Compute the cosine similarity. Add 1e-9 for numerical stability
    # torch.mv : Performs a matrix-vector product of the matrix input and the vector vec.
    cos = torch.mv(W, word_vector) / torch.sqrt(torch.sum(W * W, dim=1) *
                                      (torch.sum(word_vector * word_vector) + 1e-9))
    topk = torch.topk(cos, k=k+1)[1]
    for i in topk[2:]:  # 0is the <unk> with nan similarity, 1 is input word
        print(f'cosine sim={float(cos[i]):.3f}: {data.idx_to_token[i]}')

In [47]:
def knn(k, word, data):
    word_vector = word
    W = torch.tensor(data.idx_to_vec)
    # Compute the cosine similarity. Add 1e-9 for numerical stability
    # torch.mv : Performs a matrix-vector product of the matrix input and the vector vec.
    cos = torch.mv(W, word_vector) / torch.sqrt(torch.sum(W * W, dim=1) *
                                      (torch.sum(word_vector * word_vector) + 1e-9))
    topk = torch.topk(cos, k=k+1)[1]
    return topk[2:], [cos[int(i)] for i in topk[2:]]

In [40]:
get_similar_tokens(5, 'man', glove_6b50d)

cosine sim=0.886: woman
cosine sim=0.856: boy
cosine sim=0.845: another
cosine sim=0.837: old


/tmp/ipykernel_81253/1667884341.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  word_vector = torch.tensor(data.idx_to_vec[data.token_to_idx[word]])
/tmp/ipykernel_81253/1667884341.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  W = torch.tensor(data.idx_to_vec)


In [42]:
knn(5, 'man', glove_6b50d)

/tmp/ipykernel_81253/3120575760.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  word_vector = torch.tensor(data.idx_to_vec[data.token_to_idx[word]])
/tmp/ipykernel_81253/3120575760.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  W = torch.tensor(data.idx_to_vec)


(tensor([ 788, 1607,  171,  168]),
 [tensor(0.8860), tensor(0.8564), tensor(0.8453), tensor(0.8372)])

In [70]:
def get_analogy(token_a, token_b, token_c, embed):
    vecs = [embed.idx_to_vec[embed.token_to_idx[token_a]],
            embed.idx_to_vec[embed.token_to_idx[token_b]],
            embed.idx_to_vec[embed.token_to_idx[token_c]]
            ]
    x = vecs[1] - vecs[0] + vecs[2]
    topk, cos = knn(3, x, embed)
    return embed.idx_to_token[int(topk[0])]  # Remove unknown words

In [66]:
get_analogy('man', 'woman', 'son', glove_6b50d)

/tmp/ipykernel_81253/1558377015.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  W = torch.tensor(data.idx_to_vec)


'mother'

In [67]:
get_analogy('beijing', 'china', 'tokyo', glove_6b50d)

/tmp/ipykernel_81253/1558377015.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  W = torch.tensor(data.idx_to_vec)


'tokyo'

In [68]:
get_analogy('bad', 'worst', 'big', glove_6b50d)

/tmp/ipykernel_81253/1558377015.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  W = torch.tensor(data.idx_to_vec)


'worst'

In [69]:
get_analogy('do', 'did', 'go', glove_6b50d)

/tmp/ipykernel_81253/1558377015.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  W = torch.tensor(data.idx_to_vec)


'before'